*Теоретический материал:* https://youtu.be/kSFrLjBWbd4?t=153

*Разбор данного раздела:* https://youtu.be/kSFrLjBWbd4?t=4447

In [ ]:
from flask import Flask # Импортируем модуль Flask 

In [ ]:
app = Flask(__name__) # Создаем экземпляр Flask (аргумнтом передается имя модуля или пакета)

In [ ]:
@app.route('/') # Декоратор, указывает, что URL будет вызывать нашу функцию
def hello(): # Задаем функцию для вывода сообщения
    return 'Hello, WORLD!!!'

In [ ]:
import threading # Импортируем модуль threading для запуска в отдельном потоке

In [ ]:
threading.Thread(target = app.run, kwargs = {'host' : 'localhost', 'port':5000}).start() # Запускаем в отдельном потоке метод app.run, который запускает локальный сервер

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


In [ ]:
import requests # Библиотека для HTTP-запросов

In [ ]:
r = requests.get('http://localhost:5000') # Запрашиваем данные с локального сервера по порту 5000
print(r.status_code) # Статус ответа
print(r.encoding) # Кодировка после декодирования
print(r.apparent_encoding) # Текущая кодировка
print(r.text) # Соержимое страницы

127.0.0.1 - - [09/Jan/2021 22:58:37] "GET / HTTP/1.1" 200 -


200
utf-8
ascii
Hello, WORLD!!!


127.0.0.1 - - [09/Jan/2021 22:58:39] "GET / HTTP/1.1" 200 -


# Keras & Flask

*Разбор данного раздела:* https://youtu.be/kSFrLjBWbd4?t=4812


In [ ]:
import tensorflow
import keras
print(tensorflow.__version__)
print(keras.__version__)

2.3.0
2.4.3


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [ ]:
# Загружаем MNIST
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Нормализуем значения пикспелей (от 0 до 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Приводим лейблы к формату One Hot
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(x_train.shape, y_train.shape)

(60000, 28, 28, 1) (60000, 10)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

# Результат модели
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


# Сохраняем саму модель
# Сериализация в JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# Сохраняем(сериализуем) веса  в формате HDF5
model.save_weights("model.h5")
print("Сохранено")

Epoch 1/5
469/469 [==============================] - 2s 5ms/step - loss: 2.2812 - accuracy: 0.1519 - val_loss: 2.2500 - val_accuracy: 0.2837
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 2.2332 - accuracy: 0.2494 - val_loss: 2.1924 - val_accuracy: 0.4636
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 2.1724 - accuracy: 0.3481 - val_loss: 2.1154 - val_accuracy: 0.5784
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 2.0909 - accuracy: 0.4253 - val_loss: 2.0095 - val_accuracy: 0.6441
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 1.9780 - accuracy: 0.4875 - val_loss: 1.8663 - val_accuracy: 0.6905
Test loss: 1.8662962913513184
Test accuracy: 0.690500020980835
Сохранено


In [ ]:
from flask import Flask, request # request - это объекты, которыми управляет flask handles (get, set, post и т.д.)
from flask import render_template # этот объект автоматически зарендерит нам шаблон html
import imageio # для работы с изображениями
import numpy as np
import tensorflow.keras.models
import re # регулярные питоновские выражения
from tensorflow.keras.models import model_from_json
import sys 
import os
from tensorflow.keras.preprocessing import image
# Задаем имя серверу
app = Flask(__name__)

import tensorflow as tf
import base64 # универсальный формат кодирования (часто используется с изображениями)


In [ ]:
# Открываем json файл разметки модели
json_file = open('model.json','r') 
loaded_model_json = json_file.read() # считываем
json_file.close() # закрываем
loaded_model = model_from_json(loaded_model_json) # используем керас, чтобы считать разметку архитектуры
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [ ]:
loaded_model.load_weights("model.h5") # подгружаем веса
print("Загружено с диска")

Загружено с диска


In [ ]:
# компилируем модель
loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# оценим ее точность еще раз
loss,accuracy = loaded_model.evaluate(x_test,y_test)
print('accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 1.8663 - accuracy: 0.6905
accuracy: 0.690500020980835


In [ ]:
# декодируем изображение из сырого(бинарного) представления в его base64 
def convertImage(imgData1):
    imgstr = re.search(b'base64,(.*)',imgData1).group(1) # находим строчку, в которой содержится побитовое представление изображение,
                                                        # возвращает вторую(1) подгруппу
    with open('output.png','wb') as output: # временный файл, в который записываем декодированное представление
        output.write(base64.b64decode(imgstr))

In [ ]:
# # Указываем адрес по которому будет откликаться Web сервис и задаем саму функцию веб сервиса. Это наша стартовая страница
@app.route('/')
@app.route('/index')
def index():
    # рендерим заранее построенный HTML файл
    return render_template("index.html")

In [ ]:
# маршрут, по которому будет идти предсказание
@app.route('/predict/',methods=['GET','POST'])
def predict():
    # в любой момент, когда пользователь вызовет predict
    # после того, как нарисовал картинку, произойдет
    # inference/внедрение, вернется результат классификации
    
    # получить сырой формат изображения (этот метод объекта Flask)
    imgData = request.get_data()
    
    # кодируем в подходящий формат
    convertImage(imgData)
    print ("debug")
    # считываем изображение в память
    img = image.load_img('output.png', target_size=(28,28), color_mode = 'grayscale')
    sample = np.array(img)
    sample = np.reshape(sample, (1, 28, 28, 1))
    sample = sample.astype('float32')   
    out = loaded_model.predict(sample)
    print ("debug2")               
    print(out)
    print(np.argmax(out,axis=1))
    print ("debug3")

# преобразуем ответ в строку
    response = np.array_str(np.argmax(out,axis=1))
    return response
        # обнуляем граф, чтобы можно было ввести новые данные
    #tf.reset_default_graph()

In [ ]:
# #     # этот порт выделяем под прогонку
port = int(os.environ.get('PORT', 5003))
    
    # прогоняем на локальном сервере по заданному порту
app.run(host='127.0.0.1', port=port, threaded=False)    
    # опционально, если хотите прогнать в режиме дебага
    #app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jan/2021 23:00:08] "GET / HTTP/1.1" 200 -


debug
debug2

127.0.0.1 - - [09/Jan/2021 23:00:10] "POST /predict/ HTTP/1.1" 200 -



[[1.0000000e+00 0.0000000e+00 6.0081064e-38 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 6.1639708e-31 0.0000000e+00]]
[0]
debug3
